In [1]:
import pandas as pd
import requests
import os
import json
import time


In [14]:
# Constants
M2M_API_URL = "https://m2m.cr.usgs.gov/api/api/json/stable"  # Base URL for USGS M2M API
LOGIN_ENDPOINT = f"{M2M_API_URL}/login"
SEARCH_ENDPOINT = f"{M2M_API_URL}/scene-search"
DOWNLOAD_ENDPOINT = f"{M2M_API_URL}/download"


# Load your NASA Earthdata credentials
USERNAME = "rohanth"  # Replace with your Earthdata username
PASSWORD = "fmv@azm-cvh4HKW-egm"  # Replace with your Earthdata password
API_TOKEN = "wm0DyYKik@qdHwdYloSFZwsaJPITZ0aCPljss67jttT9@dVSsSxuDDxb_pJD9uga"
jU_3jQqqfTNmuRLDchCnlJMV9eB790yDCYyJkywFKWQpm4Pr4kEZxB3oTj0IP_0M

In [15]:
# Function to authenticate and fetch data
def search_landsat_granules(latitude, longitude, platform="LANDSAT_8"):
    """
    Search Landsat granules for the given coordinates.
    """
    headers = {
        "Authorization": f"Bearer {API_TOKEN}"
    }
    params = {
        "bounding_box": f"{longitude-0.05},{latitude-0.05},{longitude+0.05},{latitude+0.05}",
        "platform": platform,
        "collection_concept_id": "C2032345237-LPCLOUD",  # Replace with actual Landsat SR collection ID
        "page_size": 1,
        "temporal": "2024-06-01T00:00:00Z,2024-06-30T23:59:59Z",  # Adjust date range if needed
    }

    response = requests.get(SEARCH_ENDPOINT, headers=headers, params=params)

    if response.status_code == 200:
        granules = response.json()
        if granules.get("feed", {}).get("entry"):
            return granules["feed"]["entry"][0]["links"]
        else:
            return None
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None


In [16]:
# Main logic
def fetch_landsat_images(csv_file, output_folder):
    """
    Fetch Landsat SR TIFF images for coordinates in a CSV file.
    """
    # Read CSV file
    data = pd.read_csv(csv_file)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for index, row in data.iterrows():
        latitude = row["latitude"]
        longitude = row["longitude"]

        print(f"Searching Landsat images for lat: {latitude}, lon: {longitude}")
        granule_links = search_landsat_granules(latitude, longitude)

        if granule_links:
            first_link = next((link for link in granule_links if "data" in link.get("rel", "")), None)
            if first_link:
                download_url = first_link["href"]
                file_name = os.path.join(output_folder, os.path.basename(download_url))

                # Download file
                print(f"Downloading: {download_url}")
                file_response = requests.get(download_url, headers={"Authorization": f"Bearer {API_TOKEN}"})
                if file_response.status_code == 200:
                    with open(file_name, "wb") as f:
                        f.write(file_response.content)
                    print(f"Saved to {file_name}")
                else:
                    print(f"Failed to download {download_url}: {file_response.status_code}")
        else:
            print(f"No granules found for lat: {latitude}, lon: {longitude}")


In [17]:
# Example usage
csv_file = "north_american_forests.csv"  # Path to your CSV file with 'latitude' and 'longitude' columns
output_folder = "landsat_images"  # Folder to save downloaded images
fetch_landsat_images(csv_file, output_folder)

Searching Landsat images for lat: 48.799, lon: -123.056
Error: 401, {"errors":["Token does not exist"]}
No granules found for lat: 48.799, lon: -123.056
Searching Landsat images for lat: 61.18, lon: -149.105
Error: 401, {"errors":["Token does not exist"]}
No granules found for lat: 61.18, lon: -149.105
Searching Landsat images for lat: 44.12, lon: -71.195
Error: 401, {"errors":["Token does not exist"]}
No granules found for lat: 44.12, lon: -71.195
Searching Landsat images for lat: 34.184, lon: -83.561
Error: 401, {"errors":["Token does not exist"]}
No granules found for lat: 34.184, lon: -83.561
Searching Landsat images for lat: 54.725, lon: -101.861
Error: 401, {"errors":["Token does not exist"]}
No granules found for lat: 54.725, lon: -101.861
Searching Landsat images for lat: 46.354, lon: -84.303
Error: 401, {"errors":["Token does not exist"]}
No granules found for lat: 46.354, lon: -84.303
Searching Landsat images for lat: 37.865, lon: -119.538
Error: 401, {"errors":["Token does n